# Projects in AI and ML HW6 - Part 1



In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and pl

In [2]:
!pip install transformers datasets evaluate torch rouge_score accelerate

from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
from datasets import concatenate_datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 876.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=93df824f62b904edaa8bec804ae62786e95d3da0d8e5af270055a0b573c56f00


I chose to use the XSum (Extreme Sumarization) dataset. It consists of news articles and is used for evaluation of abstractive single-document summarization systems. It's goal is to create short one-sentence new summary answering the question "What is the article about?". It consists of over 220,000 news articles with a one-sentence summary. It consists of articles from BBC and a wide variety of domains including news, politics, sports, weather, and more.

https://paperswithcode.com/dataset/xsum


In [3]:
# load XSum dataset
dataset = load_dataset("xsum")

# combining train and validation sets into one larger training pool
combined_data = concatenate_datasets([dataset["train"], dataset["validation"]])

subset_size = 1000

# take random subset of combined data
combined_data = combined_data.select(range(subset_size))

train_size = int(0.9 * len(combined_data))
test_size = len(combined_data) - train_size

train_data, test_data = combined_data.select(range(train_size)), combined_data.select(range(train_size, len(combined_data)))

print(f"\nTraining Samples: {len(train_data)}")
print(f"Testing Samples: {len(test_data)}")
print(f"\nExample training sample: {train_data[0]}")
print(f"Example testing sample: {test_data[0]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]


Training Samples: 900
Testing Samples: 100

Example training sample: {'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there i

In [4]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

batch_size = 2
learning_rate = 3e-5
epochs = 2
accumulation_steps = 4
max_length = 256

model.gradient_checkpointing_enable()

def collate_fn(batch):
    # tokenize documents with dynamic padding
    inputs = tokenizer(
        [x["document"] for x in batch],
        max_length=max_length,
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )

    labels = tokenizer(
        [x["summary"] for x in batch],
        max_length=max_length,
        padding="longest",
        truncation=True,
        return_tensors="pt"
    ).input_ids

    labels[labels == tokenizer.pad_token_id] = -100

    inputs = {key: val.to(device) for key, val in inputs.items()}
    labels = labels.to(device)

    return inputs, labels

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

def train_model(model, train_loader):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch, (inputs, labels) in enumerate(train_loader):
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    labels=labels
                )
                loss = outputs.loss / accumulation_steps  # normalize loss for accumulation

            # backprop
            loss.backward()

            # gradient accumulation
            if (batch + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                scheduler.step()

            total_loss += loss.item() * accumulation_steps

            if (batch + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{epochs}], Batch [{batch + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch + 1}/{epochs}], Average Loss: {avg_loss:.4f}")

        train_model(model, train_loader)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [6]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from transformers import pipeline
from evaluate import load

rouge = load("rouge")
bleu = load("bleu")

# Reinitialize Model and Tokenizer
model_name = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

model.config.use_cache = False

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=-1)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [7]:
def evaluate_model(data, num_samples=50):
    references = []
    predictions = []

    eval_data = data.select(range(min(num_samples, len(data))))

    for item in eval_data:
        try:
            if not item["document"].strip():
                print(f"Skipping empty document: {item['document']}")
                continue
            # Truncated because of memory limitations
            truncated_doc = item["document"][:1024]
            summary = summarizer(
                truncated_doc,
                max_length=50,
                min_length=25,
                do_sample=False,
                truncation=True
            )[0]["summary_text"]
            if not summary.strip():
                print(f"Generated empty summary for: {item['document']}")
                continue
            predictions.append(summary)
            references.append(item["summary"])

        except Exception as e:
            print(f"Error processing document: {item['document'][:300]}...")
            print(f"Error: {e}")
            continue

    if len(predictions) == 0 or len(references) == 0:
        print("No valid predictions or references found. Evaluation aborted.")
        return None, None

    rouge_result = rouge.compute(predictions=predictions, references=references)
    bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    print("\n==================== Evaluation Results ====================")
    print(f"ROUGE-1 Score: {rouge_result['rouge1']:.4f}")
    print(f"ROUGE-2 Score: {rouge_result['rouge2']:.4f}")
    print(f"ROUGE-L Score: {rouge_result['rougeL']:.4f}")
    print(f"BLEU Score: {bleu_result['bleu']:.4f}")
    print("============================================================")

    return rouge_result, bleu_result

evaluate_model(test_data)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 50, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



==================== Evaluation Results ====================
ROUGE-1 Score: 0.1675
ROUGE-2 Score: 0.0162
ROUGE-L Score: 0.1199
BLEU Score: 0.0000


({'rouge1': np.float64(0.16753165219056126),
  'rouge2': np.float64(0.016177896093032662),
  'rougeL': np.float64(0.1198998700995478),
  'rougeLsum': np.float64(0.1200348490893797)},
 {'bleu': 0.0,
  'precisions': [0.14025590551181102,
   0.010090817356205853,
   0.0010351966873706005,
   0.0],
  'brevity_penalty': 1.0,
  'length_ratio': 1.82078853046595,
  'translation_length': 2032,
  'reference_length': 1116})

In [11]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


# Define hyperparameters for tuning
hyperparameters = {
    "learning_rate": [1e-5, 3e-5, 5e-5],
    "batch_size": [4],
    "epochs": [2, 3],
    "accumulation_steps": [4],
    "weight_decay": [0.01, 0.001],
}

best_bleu = 0.0
best_hyperparams = {}

def evaluate_model(data):
    references = []
    predictions = []
    eval_data = data.select(range(min(50, len(data))))

    for item in eval_data:
        inputs = tokenizer(item["document"], return_tensors="pt", max_length=256, truncation=True)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=50, min_length=25, do_sample=False)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        predictions.append(summary)
        references.append(item["summary"])

    rouge_result = rouge.compute(predictions=predictions, references=references)
    bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    print(f"ROUGE-1: {rouge_result['rouge1']:.4f}, ROUGE-2: {rouge_result['rouge2']:.4f}, ROUGE-L: {rouge_result['rougeL']:.4f}")
    print(f"BLEU: {bleu_result['bleu']:.4f}")

    return rouge_result, bleu_result


In [12]:
# Tune hyperparameters
for lr in hyperparameters["learning_rate"]:
    for bs in hyperparameters["batch_size"]:
        for ep in hyperparameters["epochs"]:
            for acc in hyperparameters["accumulation_steps"]:
                for wd in hyperparameters["weight_decay"]:
                    print(f"\nTraining with lr={lr}, batch_size={bs}, epochs={ep}, accumulation_steps={acc}, weight_decay={wd}")

                    model_name = "facebook/bart-base"
                    model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
                    tokenizer = BartTokenizer.from_pretrained(model_name)

                    batch_size = bs
                    learning_rate = lr
                    epochs = ep
                    accumulation_steps = acc
                    weight_decay = wd

                    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

                    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
                    total_steps = len(train_loader) * epochs
                    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

                    def train_model(model, train_loader):
                        model.train()
                        for epoch in range(epochs):
                            total_loss = 0
                            for batch, (inputs, labels) in enumerate(train_loader):
                                inputs = {key: val.to(device) for key, val in inputs.items()}
                                labels = labels.to(device)

                                with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                                    outputs = model(
                                        input_ids=inputs["input_ids"],
                                        attention_mask=inputs["attention_mask"],
                                        labels=labels
                                    )
                                    loss = outputs.loss / accumulation_steps
                                loss.backward()
                                if (batch + 1) % accumulation_steps == 0:
                                    optimizer.step()
                                    optimizer.zero_grad()
                                    scheduler.step()

                                total_loss += loss.item() * accumulation_steps

                            avg_loss = total_loss / len(train_loader)
                            print(f"Epoch [{epoch + 1}/{epochs}], Avg Loss: {avg_loss:.4f}")

                    train_model(model, train_loader)

                    rouge_result, bleu_result = evaluate_model(test_data)

                    if bleu_result and bleu_result['bleu'] > best_bleu:
                        best_bleu = bleu_result['bleu']
                        best_hyperparams = {
                            "learning_rate": lr,
                            "batch_size": bs,
                            "epochs": ep,
                            "accumulation_steps": acc,
                            "weight_decay": wd
                        }

                        model.save_pretrained("best_bart_model")
                        tokenizer.save_pretrained("best_bart_model")

print("\n==================== Best Hyperparameters ====================")
print(f"Best BLEU Score: {best_bleu}")
print(f"Best Hyperparameters: {best_hyperparams}")
print("==============================================================")


Training with lr=1e-05, batch_size=4, epochs=2, accumulation_steps=4, weight_decay=0.01
Epoch [1/2], Avg Loss: 2.7057
Epoch [2/2], Avg Loss: 2.4307
Training complete!
ROUGE-1: 0.3029, ROUGE-2: 0.0981, ROUGE-L: 0.2408
BLEU: 0.0629


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



Training with lr=1e-05, batch_size=4, epochs=2, accumulation_steps=4, weight_decay=0.001
Epoch [1/2], Avg Loss: 2.7102
Epoch [2/2], Avg Loss: 2.4348
Training complete!
ROUGE-1: 0.2804, ROUGE-2: 0.0880, ROUGE-L: 0.2265
BLEU: 0.0529

Training with lr=1e-05, batch_size=4, epochs=3, accumulation_steps=4, weight_decay=0.01
Epoch [1/3], Avg Loss: 2.7166
Epoch [2/3], Avg Loss: 2.4277
Epoch [3/3], Avg Loss: 2.3227
Training complete!
ROUGE-1: 0.2918, ROUGE-2: 0.0857, ROUGE-L: 0.2305
BLEU: 0.0548

Training with lr=1e-05, batch_size=4, epochs=3, accumulation_steps=4, weight_decay=0.001
Epoch [1/3], Avg Loss: 2.6883
Epoch [2/3], Avg Loss: 2.4096
Epoch [3/3], Avg Loss: 2.3105
Training complete!
ROUGE-1: 0.2992, ROUGE-2: 0.1044, ROUGE-L: 0.2517
BLEU: 0.0665


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



Training with lr=3e-05, batch_size=4, epochs=2, accumulation_steps=4, weight_decay=0.01
Epoch [1/2], Avg Loss: 2.6256
Epoch [2/2], Avg Loss: 2.2418
Training complete!
ROUGE-1: 0.2951, ROUGE-2: 0.0900, ROUGE-L: 0.2325
BLEU: 0.0499

Training with lr=3e-05, batch_size=4, epochs=2, accumulation_steps=4, weight_decay=0.001
Epoch [1/2], Avg Loss: 2.6245
Epoch [2/2], Avg Loss: 2.2433
Training complete!
ROUGE-1: 0.3283, ROUGE-2: 0.1162, ROUGE-L: 0.2597
BLEU: 0.0879


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



Training with lr=3e-05, batch_size=4, epochs=3, accumulation_steps=4, weight_decay=0.01
Epoch [1/3], Avg Loss: 2.6315
Epoch [2/3], Avg Loss: 2.2438
Epoch [3/3], Avg Loss: 2.0067
Training complete!
ROUGE-1: 0.3075, ROUGE-2: 0.0862, ROUGE-L: 0.2445
BLEU: 0.0519

Training with lr=3e-05, batch_size=4, epochs=3, accumulation_steps=4, weight_decay=0.001
Epoch [1/3], Avg Loss: 2.6301
Epoch [2/3], Avg Loss: 2.2331
Epoch [3/3], Avg Loss: 2.0384
Training complete!
ROUGE-1: 0.3012, ROUGE-2: 0.1095, ROUGE-L: 0.2432
BLEU: 0.0739

Training with lr=5e-05, batch_size=4, epochs=2, accumulation_steps=4, weight_decay=0.01
Epoch [1/2], Avg Loss: 2.6339
Epoch [2/2], Avg Loss: 2.1289
Training complete!
ROUGE-1: 0.3049, ROUGE-2: 0.1146, ROUGE-L: 0.2465
BLEU: 0.0792

Training with lr=5e-05, batch_size=4, epochs=2, accumulation_steps=4, weight_decay=0.001
Epoch [1/2], Avg Loss: 2.6299
Epoch [2/2], Avg Loss: 2.1096
Training complete!
ROUGE-1: 0.3210, ROUGE-2: 0.1062, ROUGE-L: 0.2709
BLEU: 0.0758

Training with

From the hyperparameter tuning, the best hyperparameters were a learning rate of 3e-05, a batch size of 4, 2 epochs, 4 accumulation steps, and 0.001 weight decay. I did not include more options for batch size and accumulation steps due to how long it took the program to run, but I determined that these were the optimal hyperparameters. However, in the future, I would like to include more hyperparameters to tune with more values. The model acheived a BLEU score of 0.879, which is the highest of the tested options and fairly decent. The ROUGE scores were also decent, showing improvement in model performance compared with other previous experiments. However, I would have expected more epochs to have better results, so in the future I would test this with other values as the combination of hyperparameters could also affect the result.

# Part 2: Reinforcement Learning
##Task 1:
We discussed how we can formulate RL problems as an MDP. Describe any
real-world application that can be formulated as an MDP. Describe the state space, action
space, transition model, and rewards for that problem. You do not need to be precise in the
description of the transition model and reward (no formula is needed). Qualitative description
is enough.


###Example: Autonomous Driving

Autonomous vehicles (self-driving cars) need to navigate complex environments and make real-time decisions to ensure safety and efficiency, so it can modeled as a Markov Decision Process (MDP). The objective of the MDP is to maximize the cumulative reward by selecting actions that lead to safe and efficient driving, ensuring the vehicle reaches its destination without collisions or violations.

State Space (S): The state represents the environment and the vehicle's current situations. This could include the position on the road, the velocity and acceleration, the distance from other vehicles, information about the lanes on the road, status of traffic lights, presence of pedestrians or other things on the road, the conditions of the road, the time of day, etc.  

Action Space (A): The action space defines the actions the vehicle can make at each time step. These include accelerating (increase speed),  braking (slowing down), steering left or right (change lane or adjust position on the road), maintaining speed, and stopping.

Transition model (T): This defines how the environment changes in response to the chosen action. It gives the probability of transitioning from one state to another given an action. Some examples are deterministic transitions, which would include actions like maintaining speed or changing lanes having predictable outcomes. The other side is stochastic transitions, which include factors like other drivers' behavior, pedestrian behavior, and weather conditions that introduce uncertainty.

Rewards (R): This evaluates the outcome of an action to ensure safe and efficient driving. This includes both positive and negative rewards.   
Positive Rewards: successfully and safely reaching the destination
Negative Rewards: car accidents, delayed arrival, harsh braking, traffic violations


## Task 3:
RL is used in various sectors - Healthcare, recommender systems and trading
are a few of those. Pick one of the three areas. Explain one of the problems in any of these
domains that can be more effectively solved by reinforcement learning. Find an open-source
project (if any) that has addressed this problem. Explain this project in detail.

Reinforcement Learning is used in various sectors including healthcare. One problem in healthcare that can be more effectively solved by reinforcement learning is optimizing personalized treatment plans. This is one of the challenges in healthcare because personalized plans need to adapt dynamically over time based on the condition of the patient, which could change. It needs to consider the effects of past treatments as well as a number of factors that is specific to each patient including age and past medical history. Traditional models would be much less able to adapt to be personalized to each patient, because everyone's medical history and situation is different. Therefore, RL can be used to solve this problem. By training an RL agent with historical patient data, the model can learn to optimize long-term outcomes, considering potential delayed effects of treatments. Modeling this problem as an RL problem, the state space would represent the patients current health condition, medical history and treatements, the action space would be different treatments and medications that can be applied at each step, and the reward would be positive when the patients condition improves or negative when there is lack of improvement or detrimental side effects.  

An open source project that has addressed this problem is AI Clinician (Sepsis Treatement). https://github.com/matthieukomorowski/AI_Clinician

This project is titled AI Clinician – A Reinforcement Learning Agent for Sepsis Treatment. This addresses the Sepsis condition, which is a life-threatening condition caused by the body's response to infection. Optimal treatments include administering fluids and vasopressors, but determining the right dosage and timing is difficult due to individual variability in patient responses. This AI Clinician project focuses on using RL to learn optimal treatment strategies for sepsis patients in the ICU. It uses clinical data containing ICU patient records. It uses the MIMIC-III dataset. The state space is the patient state representation, and it includes the demographic information, vital signs, lab results, treatement information, and sequential data over time to capture temporal patterns. The action space contains combinations of different fluid administration dosages and vasopressor dosage levels. The reward function reflects patient outcomes, including positive reward for survival and improvement of conditions, and negative rewards for deterioration, organ failure, or death. The AI Clinician project performed better than standard clinical practice by recommending more personalized and effective treatment decisions, showing how RL can be used to more effectively solve problems like this in healthcare.

